In [1]:
import pandas as pd
import numpy as np

In [2]:
# import data
productDF = pd.read_csv('../data/lazada/clean/product.csv')
reviewDF = pd.read_csv('../data/lazada/clean/review.csv')

In [3]:
# aggregate
productRatingDF = reviewDF[['sku', 'rating']].copy()
productRatingDF['rating_count'] = productRatingDF['rating']
productRatingDF['rating_mean'] = productRatingDF['rating']
productRatingDF['rating_sum'] = productRatingDF['rating']
productRatingDF = productRatingDF[['sku', 'rating_sum', 'rating_count', 'rating_mean']] \
    .groupby('sku') \
    .agg({
        'rating_sum': sum,
        'rating_count': len,
        'rating_mean': np.mean
    }) \
    .sort_values('rating_sum', ascending=False) \
    .reset_index()
productRatingDF.head(5)

,sku,rating_sum,rating_count,rating_mean
0,SA862ELAA1JXU7ANTH,6185,1352,4.574704
1,UN355ELAA4RTXAANTH,3928,1021,3.847209
2,SA862ELAA1JXU6ANTH,3672,810,4.533333
3,UN355ELAA7LEZKANTH,2617,708,3.696328
4,SA861ELAAFIM5GANTH,1833,479,3.826722


In [4]:
# normalize rating
productRatingDF['rating_sum_rank'] = productRatingDF['rating_sum'].rank(pct=True)
productRatingDF['rating_count_rank'] = productRatingDF['rating_count'].rank(pct=True)
productRatingDF['rating_mean_rank'] = productRatingDF['rating_mean'].rank(pct=True)

# weighted sum percentile
w_count = 1
w_sum = 1
w_mean = 1
productRatingDF['final_score'] = \
    w_sum * productRatingDF['rating_sum_rank'] \
    + w_mean * productRatingDF['rating_mean_rank'] \
    + w_count * productRatingDF['rating_count_rank']

In [5]:
# explore ranking
productDF[['sku', 'name']] \
    .merge(productRatingDF.sort_values('final_score', ascending=False)) \
    .sort_values('final_score', ascending=False) \
    .drop(['sku'], axis=1) \
    .head(10)

,name,rating_sum,rating_count,rating_mean,rating_sum_rank,rating_count_rank,rating_mean_rank,final_score
5834,WARRIX SPORT กางเกงวอร์ม กีฬา WP-1707 สีดำ,509,108,4.712963,0.993371,0.990598,0.767451,2.751420
1073,Fantech Gaming Mousepad แผ่นรองเมาส์แบบสปีด สำ...,329,68,4.838235,0.983766,0.976123,0.787879,2.747768
4564,"Dell MS116 USB Optical Mouse 1000dpi, 1.8m (Bl...",477,102,4.676471,0.992695,0.989583,0.765016,2.747294
2723,Kingston Micro SD Class 10 U1 80/10 MB/s - 16G...,894,194,4.608247,0.997294,0.996753,0.744183,2.738231
2721,Sandisk MicroSD Ultra Class 10 48MB/S - 32GB(S...,6185,1352,4.574704,1.000000,1.000000,0.736878,2.736878
1515,Canon Lens EF 50mm f/1.8 STM,558,120,4.650000,0.994251,0.991951,0.748309,2.734510
424,[สายหนา 4 mm] สายชาร์จ Aukey Quick Charge 3.0+...,270,56,4.821429,0.976461,0.967938,0.784767,2.729167
2730,Sandisk MicroSD Ultra Class 10 48MB/S - 16GB(S...,3672,810,4.533333,0.999729,0.999729,0.729302,2.728761
269,Eloop E14 Power Bank 20000mAh (สีดำ) ฟรี ซองกำ...,1440,318,4.528302,0.999053,0.998782,0.728896,2.726732
4625,Wacom Intuos Draw Small (CTL-490/B0-C) - Mintblue,404,87,4.643678,0.989989,0.985119,0.747565,2.722673


In [6]:
# productRatingDF.to_csv('popular/util.csv', index=False)